In [5]:
-- Proposition 1: List all unique U.S. and Canadian customers with names and territory (limit to 20 results)
SELECT TOP 20 CustomerID, FirstName, LastName, TerritoryName
FROM (
    SELECT c.CustomerID, p.FirstName, p.LastName, t.Name AS TerritoryName
    FROM Sales.Customer AS c
    JOIN Person.Person AS p ON c.PersonID = p.BusinessEntityID
    JOIN Sales.SalesTerritory AS t ON c.TerritoryID = t.TerritoryID
    WHERE c.TerritoryID = 1  -- US
    UNION  -- remove duplicates between US & Canada
    SELECT c.CustomerID, p.FirstName, p.LastName, t.Name AS TerritoryName
    FROM Sales.Customer AS c
    JOIN Person.Person AS p ON c.PersonID = p.BusinessEntityID
    JOIN Sales.SalesTerritory AS t ON c.TerritoryID = t.TerritoryID
    WHERE c.TerritoryID = 2  -- Canada
) AS Combined
ORDER BY CustomerID;


(20 rows affected)

Total execution time: 00:00:00.145

CustomerID,FirstName,LastName,TerritoryName
11012,Lauren,Walker,Northwest
11013,Ian,Jenkins,Northwest
11014,Sydney,Bennett,Northwest
11021,Destiny,Wilson,Northwest
11022,Ethan,Zhang,Northwest
11040,Jesse,Murphy,Northwest
11062,Noah,Powell,Northwest
11063,Angela,Murphy,Northwest
11066,Grace,Butler,Northwest
11067,Caleb,Carter,Northwest


In [6]:
-- Proposition 2: Show all U.S. and Canadian customers including duplicates (TOP 20)
SELECT TOP 20 CustomerID, FirstName, LastName, TerritoryName
FROM (
    SELECT c.CustomerID, p.FirstName, p.LastName, t.Name AS TerritoryName
    FROM Sales.Customer c
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    JOIN Sales.SalesTerritory t ON c.TerritoryID = t.TerritoryID
    WHERE c.TerritoryID = 1
    UNION ALL  -- keep duplicates
    SELECT c.CustomerID, p.FirstName, p.LastName, t.Name AS TerritoryName
    FROM Sales.Customer c
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    JOIN Sales.SalesTerritory t ON c.TerritoryID = t.TerritoryID
    WHERE c.TerritoryID = 2
) AS Combined
ORDER BY CustomerID;


(20 rows affected)

Total execution time: 00:00:00.015

CustomerID,FirstName,LastName,TerritoryName
11012,Lauren,Walker,Northwest
11013,Ian,Jenkins,Northwest
11014,Sydney,Bennett,Northwest
11021,Destiny,Wilson,Northwest
11022,Ethan,Zhang,Northwest
11040,Jesse,Murphy,Northwest
11062,Noah,Powell,Northwest
11063,Angela,Murphy,Northwest
11066,Grace,Butler,Northwest
11067,Caleb,Carter,Northwest


In [8]:
-- Proposition 3: People who are both customers and employees (TOP 20)
SELECT TOP 20 BusinessEntityID, FirstName, LastName
FROM (
    SELECT p.BusinessEntityID, p.FirstName, p.LastName
    FROM Sales.Customer c
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    INTERSECT
    SELECT e.BusinessEntityID, p.FirstName, p.LastName
    FROM HumanResources.Employee e
    JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
) AS Overlap
ORDER BY BusinessEntityID;



(0 rows affected)

Total execution time: 00:00:00.006

BusinessEntityID,FirstName,LastName


In [9]:
-- Proposition 4: Employees with mailing addresses in Washington (TOP 20)
SELECT TOP 20 BusinessEntityID, FirstName, LastName, City
FROM (
    SELECT e.BusinessEntityID, p.FirstName, p.LastName, a.City
    FROM HumanResources.Employee e
    JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
    JOIN Person.BusinessEntityAddress bea ON e.BusinessEntityID = bea.BusinessEntityID
    JOIN Person.Address a ON bea.AddressID = a.AddressID
    INTERSECT
    SELECT e.BusinessEntityID, p.FirstName, p.LastName, a.City
    FROM HumanResources.Employee e
    JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
    JOIN Person.BusinessEntityAddress bea ON e.BusinessEntityID = bea.BusinessEntityID
    JOIN Person.Address a ON bea.AddressID = a.AddressID
    WHERE a.StateProvinceID = 79  -- Washington
) AS WAEmployees
ORDER BY BusinessEntityID;



(20 rows affected)

Total execution time: 00:00:00.058

BusinessEntityID,FirstName,LastName,City
1,Ken,Sánchez,Newport Hills
2,Terri,Duffy,Renton
3,Roberto,Tamburello,Redmond
5,Gail,Erickson,Bellevue
6,Jossef,Goldberg,Renton
7,Dylan,Miller,Kenmore
8,Diane,Margheim,Everett
9,Gigi,Matthew,Bellevue
11,Ovidiu,Cracium,Kenmore
12,Thierry,D'Hers,Bothell


In [10]:
-- Proposition 5: Products that have never been ordered (TOP 20)
SELECT TOP 20 ProductID, ProductName, Subcategory, Category
FROM (
    SELECT p.ProductID, p.Name AS ProductName, ps.Name AS Subcategory, pc.Name AS Category
    FROM Production.Product p
    LEFT JOIN Production.ProductSubcategory ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
    LEFT JOIN Production.ProductCategory pc ON ps.ProductCategoryID = pc.ProductCategoryID
    EXCEPT
    SELECT p.ProductID, p.Name AS ProductName, ps.Name AS Subcategory, pc.Name AS Category
    FROM Production.Product p
    JOIN Sales.SalesOrderDetail d ON p.ProductID = d.ProductID
    LEFT JOIN Production.ProductSubcategory ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
    LEFT JOIN Production.ProductCategory pc ON ps.ProductCategoryID = pc.ProductCategoryID
) AS Unordered
ORDER BY ProductID;


(20 rows affected)

Total execution time: 00:00:00.043

ProductID,ProductName,Subcategory,Category
1,Adjustable Race,NULL,NULL
2,Bearing Ball,NULL,NULL
3,BB Ball Bearing,NULL,NULL
4,Headset Ball Bearings,NULL,NULL
316,Blade,NULL,NULL
317,LL Crankarm,NULL,NULL
318,ML Crankarm,NULL,NULL
319,HL Crankarm,NULL,NULL
320,Chainring Bolts,NULL,NULL
321,Chainring Nut,NULL,NULL


In [11]:
-- Proposition 6: Customers who never placed any orders (TOP 20)
SELECT TOP 20 CustomerID, FirstName, LastName
FROM (
    SELECT c.CustomerID, p.FirstName, p.LastName
    FROM Sales.Customer c
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    EXCEPT
    SELECT c.CustomerID, p.FirstName, p.LastName
    FROM Sales.Customer c
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    JOIN Sales.SalesOrderHeader h ON c.CustomerID = h.CustomerID
) AS Inactive
ORDER BY CustomerID;


(0 rows affected)

Total execution time: 00:00:00.116

CustomerID,FirstName,LastName


In [12]:
-- Proposition 7: Combine vendors and employees (TOP 20)
SELECT TOP 20 BusinessEntityID, FirstName, LastName, EntityType
FROM (
    SELECT v.BusinessEntityID, p.FirstName, p.LastName, 'Vendor' AS EntityType
    FROM Purchasing.Vendor v
    JOIN Person.Person p ON v.BusinessEntityID = p.BusinessEntityID
    UNION
    SELECT e.BusinessEntityID, p.FirstName, p.LastName, 'Employee' AS EntityType
    FROM HumanResources.Employee e
    JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
) AS AllEntities
ORDER BY BusinessEntityID;


(20 rows affected)

Total execution time: 00:00:00.010

BusinessEntityID,FirstName,LastName,EntityType
1,Ken,Sánchez,Employee
2,Terri,Duffy,Employee
3,Roberto,Tamburello,Employee
4,Rob,Walters,Employee
5,Gail,Erickson,Employee
6,Jossef,Goldberg,Employee
7,Dylan,Miller,Employee
8,Diane,Margheim,Employee
9,Gigi,Matthew,Employee
10,Michael,Raheem,Employee


In [16]:
-- Proposition 8: Customers and employees sharing same city/postal code (TOP 20)
SELECT TOP 20 City, PostalCode, CustomerCount
FROM (
    SELECT a.City, a.PostalCode, COUNT(*) AS CustomerCount
    FROM Person.Address a
    JOIN Person.BusinessEntityAddress bea ON a.AddressID = bea.AddressID
    JOIN Sales.Customer c ON bea.BusinessEntityID = c.PersonID
    GROUP BY a.City, a.PostalCode
    INTERSECT
    SELECT a.City, a.PostalCode, COUNT(*) AS EmployeeCount
    FROM Person.Address a
    JOIN Person.BusinessEntityAddress bea ON a.AddressID = bea.AddressID
    JOIN HumanResources.Employee e ON bea.BusinessEntityID = e.BusinessEntityID
    GROUP BY a.City, a.PostalCode
) AS Overlaps
ORDER BY City, PostalCode;


(0 rows affected)

Total execution time: 00:00:00.098

City,PostalCode,CustomerCount


In [15]:
-- Proposition 9: Employees not in Sales (TOP 20)
SELECT TOP 20 BusinessEntityID, FirstName, LastName
FROM (
    SELECT e.BusinessEntityID, p.FirstName, p.LastName
    FROM HumanResources.Employee e
    JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
    EXCEPT
    SELECT e.BusinessEntityID, p.FirstName, p.LastName
    FROM HumanResources.Employee e
    JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
    JOIN HumanResources.EmployeeDepartmentHistory dh ON e.BusinessEntityID = dh.BusinessEntityID
    JOIN HumanResources.Department d ON dh.DepartmentID = d.DepartmentID
    WHERE d.Name = 'Sales'
) AS NonSales
ORDER BY BusinessEntityID;


(20 rows affected)

Total execution time: 00:00:00.032

BusinessEntityID,FirstName,LastName
1,Ken,Sánchez
2,Terri,Duffy
3,Roberto,Tamburello
4,Rob,Walters
5,Gail,Erickson
6,Jossef,Goldberg
7,Dylan,Miller
8,Diane,Margheim
9,Gigi,Matthew
10,Michael,Raheem


In [17]:
-- Proposition 10: High freight or high subtotal orders (TOP 20)
SELECT TOP 20 SalesOrderID, CustomerID, FirstName, LastName, Category, Freight, SubTotal
FROM (
    SELECT h.SalesOrderID, c.CustomerID, p.FirstName, p.LastName, 'High Freight' AS Category, h.Freight, h.SubTotal
    FROM Sales.SalesOrderHeader h
    JOIN Sales.Customer c ON h.CustomerID = c.CustomerID
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    WHERE h.Freight > 500
    UNION
    SELECT h.SalesOrderID, c.CustomerID, p.FirstName, p.LastName, 'High Subtotal' AS Category, h.Freight, h.SubTotal
    FROM Sales.SalesOrderHeader h
    JOIN Sales.Customer c ON h.CustomerID = c.CustomerID
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    WHERE h.SubTotal > 20000
) AS BigOrders
ORDER BY SalesOrderID;



(20 rows affected)

Total execution time: 00:00:00.025

SalesOrderID,CustomerID,FirstName,LastName,Category,Freight,SubTotal
43659,29825,James,Hendergart,High Freight,616.0984,20565.6206
43659,29825,James,Hendergart,High Subtotal,616.0984,20565.6206
43661,29734,Jauna,Elson,High Freight,985.553,32726.4786
43661,29734,Jauna,Elson,High Subtotal,985.553,32726.4786
43662,29994,Robin,McGuigan,High Freight,867.2389,28832.5289
43662,29994,Robin,McGuigan,High Subtotal,867.2389,28832.5289
43664,29898,Sandeep,Katyal,High Freight,732.81,24432.6088
43664,29898,Sandeep,Katyal,High Subtotal,732.81,24432.6088
43668,29614,Ryan,Calafato,High Freight,1081.8017,35944.1562
43668,29614,Ryan,Calafato,High Subtotal,1081.8017,35944.1562
